#LIMPIEZA DE DATOS Y NORMALIZAMOS EL ARCHIVO, dejando los años en una columnna.

In [34]:
import os
import pandas as pd

# --- Primera Parte: Limpieza y Normalización ---

# Definir la ruta base para tus archivos csv/xlsx
ruta_base = "C:\\Users\\santi\\Challenge-Analytics_Engineer\\DB\\Archivos\\"  # Cambiá esto a la ruta donde están los archivos

# Lista de archivos Excel a procesar
archivos_excel = ["Origen.xlsx"]

# Función para limpiar un archivo Excel
def limpiar_archivo_excel(nombre_archivo):
    ruta_archivo = os.path.join(ruta_base, nombre_archivo)  # Crear la ruta completa
    
    try:
        # Leer el archivo Excel
        df = pd.read_excel(ruta_archivo, engine='openpyxl')  # Aseguramos el uso de openpyxl
        
        # Renombrar columnas
        df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_')
        
        # Rellenar valores nulos
        df.fillna(0, inplace=True)
        
        # Asegurar que los números sean float
        for col in df.select_dtypes(include=['object']):
            try:
                df[col] = df[col].str.replace('%', '').str.replace(',', '').astype(float)
            except ValueError:
                pass  # Ignorar columnas no convertibles
        
        # Guardar archivo limpio como CSV
        nombre_limpio = f"limpio_{os.path.splitext(nombre_archivo)[0]}.csv"
        ruta_guardado = os.path.join(ruta_base, nombre_limpio)
        df.to_csv(ruta_guardado, index=False, encoding='utf-8')
        print(f"Archivo limpio guardado: {ruta_guardado}")
    
    except FileNotFoundError:
        print(f"Archivo no encontrado: {ruta_archivo}")
    except Exception as e:
        print(f"Error procesando {nombre_archivo}: {e}")

# Limpiar cada archivo Excel
for archivo in archivos_excel:
    limpiar_archivo_excel(archivo)

# --- Segunda Parte: Transformar el formato ---

# Ruta del archivo CSV limpio
ruta_csv = "C:\\Users\\santi\\Challenge-Analytics_Engineer\\DB\\Archivos\\limpio_Origen.csv"  # Cambiá esto por la ruta de tu archivo limpio

# Leer el archivo CSV limpio
df = pd.read_csv(ruta_csv)

# Transformar el formato ancho a largo
df_long = df.melt(
    id_vars=["pais", "pais_id", "indicador", "id_serie"],  # Columnas constantes
    var_name="anio",  # Nueva columna para los años
    value_name="valor"  # Nueva columna para los valores
)

# Limpiar la columna de años (eliminar texto adicional y dejar solo el número del año)
df_long["anio"] = df_long["anio"].str.extract(r"(\d{4})").astype(int)

# Guardar el archivo transformado
ruta_salida = "C:\\Users\\santi\\Challenge-Analytics_Engineer\\DB\\Archivos\\normalizado_Origen.csv"  # Ruta completa del archivo de salida
df_long.to_csv(ruta_salida, index=False, encoding="utf-8")

print(f"Archivo transformado guardado en: {ruta_salida}")


Archivo limpio guardado: C:\Users\santi\Challenge-Analytics_Engineer\DB\Archivos\limpio_Origen.csv
Archivo transformado guardado en: C:\Users\santi\Challenge-Analytics_Engineer\DB\Archivos\normalizado_Origen.csv


#AHORA STANDARIZAMOS LOS INDICADORES

In [18]:
import pandas as pd

# Ruta del archivo CSV normalizado
ruta_csv_normalizado = "C:\\Users\\santi\\Challenge-Analytics_Engineer\\DB\\Archivos\\Origen_normalizado.xlsx"  # Cambiá esto por tu ruta

# Leer el archivo normalizado
df_normalizado = pd.read_csv(ruta_csv_normalizado)

# Diccionario para renombrar las columnas de indicadores (completo)
renombrar_columnas = {
    "Fixed broadband subscriptions": "banda_ancha_fija",
    "Fixed telephone subscriptions (per 100 people)": "lineas_telefonicas_por_100_personas",
    "GDP per capita (current US$)": "pib_per_capita_usd",
    "ICT goods exports (% of total goods exports)": "exportaciones_tic_pct",
    "ICT goods imports (% total goods imports)": "importaciones_tic_pct",
    "Individuals using the Internet (% of population)": "usuarios_internet_pct",
    "Individuals using the Internet, female (% of female population)": "internet_mujeres_pct",
    "Individuals using the Internet, male (% of male population)": "internet_hombres_pct",
    "Mobile cellular subscriptions": "celulares_moviles",
    "Mobile cellular subscriptions (per 100 people)": "celulares_por_100_personas",
    "Secure Internet servers (per 1 million people)": "servidores_internet_por_millon",
    "Urban population (% of total population)": "poblacion_urbana_pct",
    "Government expenditure on education, total (% of GDP)": "gasto_educacion_pib_pct",
    "Research and development expenditure (% of GDP)": "gasto_investigacion_pib_pct",
    "Literacy rate, adult total (% of people ages 15 and above)": "tasa_alfabetizacion_adultos",
    "School enrollment, tertiary (% gross)": "matricula_educacion_terciaria_pct",
    "Gini index (World Bank estimate)": "indice_gini",
    "Poverty headcount ratio at national poverty lines (% of population)": "tasa_pobreza_nacional_pct",
    "Unemployment, total (% of total labor force)": "tasa_desempleo_total_pct"
}

# Pivotar los datos: Indicadores como columnas
df_pivot = df_normalizado.pivot_table(
    index=["pais", "cod_pais", "anio"],  # Claves para las filas
    columns="indicador",                   # Indicadores como columnas
    values="valor",                        # Valores de los indicadores
    aggfunc="first"                        # Usar el primer valor en caso de duplicados
)

# Renombrar las columnas con el diccionario
df_pivot.rename(columns=renombrar_columnas, inplace=True)

# Resetear el índice para que el DataFrame quede en formato tabular
df_pivot.reset_index(inplace=True)

# Ruta para guardar el archivo final
ruta_csv_renombrado = "C:\\Users\\santi\\Challenge-Analytics_Engineer\\DB\\standarizado_renombrado_Origen.csv"  # Cambiá esto por tu ruta

# Guardar el archivo CSV
df_pivot.to_csv(ruta_csv_renombrado, index=False, encoding="utf-8")

print(f"Archivo renombrado y guardado en: {ruta_csv_renombrado}")

KeyError: 'pais'